## Projet Python pour la data science (2023-2024)
*Thomas Chen, Félix de Champs, David Premachandra*  

  
Notre projet à pour objectif de donner la probabilité qu'un feu de forêt se déclenche dans le Sud-Est de la France. 
En effet, les feux de forêt sont un problème qui touche de plus en plus le territoire français, notamment à cause du réchauffement climatique qui contribue à assécher la végétation. Pour cela nous allons utiliser les données météos fournies par Météo France pour le quart Sud-Est et sur la BDIFF (Base de Données sur les Incendies de Forêt en France) se définie commen une application internet chargée de centraliser l'ensemble des données sur les incendies de forêt sur le territoire français depuis 2006 et de mettre l'ensemble de cette information à disposition du public et des services de l'Etat.


## I/ Récupération et pré-traitement des données
Les données sont toutes rassemblées dans un dossier nommé "data".

- On commence par importer les modules pour le traitement des données : pandas pour manipuler les dataframes, numpy pour faire des opérations mathématiques et datetime pour la manipulation des dates.

In [ ]:
import pandas as pd 
import numpy as np 
from datetime import datetime

- Ensuite on va charger les différentes bases de données sur python en les faisant passer du format csv au format dataframe de pandas:  
  
  - la base de données sur les incendies de forêt est disponible via ce [lien](https://bdiff.agriculture.gouv.fr/incendies).  
  - la base de données météo est disponible via ce [lien](https://meteonet.umr-cnrm.fr/dataset/data/SE/ground_stations/).
  - la base de données sur les communes est disponible via ce [lien](https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25).


In [2]:
# Dataframe de la base de données sur les incendies: 
df_incendies = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\Incendies.csv", sep = ';')

# Dataframe avec la base de données meteonet pour le quart sud-est :
df_meteo_SE = pd.read_csv(r'C:\Users\felix\Desktop\ensae-prog2A\data\meteonet\ground_stations\SE2018.csv')

# Dataframe avec juste les stations et leurs coordonnées GPS :
df_stations_SE = df_meteo_SE[['number_sta','lat','lon']]
df_stations_SE = df_stations_SE.drop_duplicates(subset='number_sta').reset_index()

# Dataframe avec la base de données permettant de faire la correspondance commune-coordonnées GPS :
df_communes = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\communes.csv")
df_communes = df_communes[['code_commune_INSEE','latitude','longitude','nom_region']]
df_communes.columns = ['code_commune_INSEE', 'lat', 'lon', 'region']
# On se restrenint aux régions qui nous intéresses
liste_regions = ['Corse', "Provence-Alpes-Côte d'Azur", "Occitanie", "Auvergne-Rhône-Alpes", 'Nouvelle-Aquitaine']
df_communes = df_communes[df_communes['region'].isin(liste_regions)].drop_duplicates(subset = 'code_commune_INSEE').reset_index()
# Correction des codes communes INSEE des départements en 0X 
for i in df_communes.index :
    if len(df_communes['code_commune_INSEE'][i])==4 :
        df_communes['code_commune_INSEE'][i] = '0'+ df_communes['code_commune_INSEE'][i]


- Les données du dataset de Météo France sont données toutes les 6 minutes pour chaque station sur l'ensemble de l'année. Nous avons donc décidé de les moyenner sur la journée afin réduire le nombre d'observations à une par jour et par station. Cela se justifie dans la mesure où les tendances intrajournalières des paramètres météorologiques n'ont pas beaucoup d'importance dans le cadre de notre modélisation.  
C'est donc le sens de ce dataframe `df_meteo_moy`.

In [3]:
df_meteo_SE['date'] = pd.to_datetime(df_meteo_SE['date'])
df_meteo_moy = df_meteo_SE.groupby(['number_sta', pd.Grouper(key = 'date', freq = 'D')]).mean().reset_index()
display(df_meteo_moy.head())

,number_sta,date,lat,lon,height_sta,dd,ff,precip,hu,td,t,psl
0,1027003,2018-01-01,45.83,5.11,196.0,NaN,NaN,NaN,80.679167,277.326250,280.587917,NaN
1,1027003,2018-01-02,45.83,5.11,196.0,NaN,NaN,NaN,76.962343,276.310084,280.177311,NaN
2,1027003,2018-01-03,45.83,5.11,196.0,NaN,NaN,NaN,80.033333,278.724167,282.273333,NaN
3,1027003,2018-01-04,45.83,5.11,196.0,NaN,NaN,NaN,93.241667,281.604583,282.673333,NaN
4,1027003,2018-01-05,45.83,5.11,196.0,NaN,NaN,NaN,82.687500,281.308333,284.290833,NaN
5,1027003,2018-01-06,45.83,5.11,196.0,NaN,NaN,NaN,85.447699,280.121548,282.566318,NaN
6,1027003,2018-01-07,45.83,5.11,196.0,NaN,NaN,NaN,91.504167,279.265833,280.565000,NaN
7,1027003,2018-01-08,45.83,5.11,196.0,NaN,NaN,NaN,90.500000,278.241250,279.699167,NaN
8,1027003,2018-01-09,45.83,5.11,196.0,NaN,NaN,NaN,87.166667,277.715417,279.842083,NaN
9,1027003,2018-01-10,45.83,5.11,196.0,NaN,NaN,NaN,83.095833,277.441250,280.148333,NaN


,number_sta,date,lat,lon,height_sta,dd,ff,precip,hu,td,t,psl
181006,84150001,2018-12-22,44.337,4.905,141.0,156.458333,0.886667,0.000000,95.516667,280.406667,281.100833,NaN
181007,84150001,2018-12-23,44.337,4.905,141.0,324.375000,3.048750,0.000000,79.316667,281.633333,285.285833,NaN
181008,84150001,2018-12-24,44.337,4.905,141.0,298.333333,5.438333,0.000833,73.445833,280.543750,285.224583,NaN
181009,84150001,2018-12-25,44.337,4.905,141.0,136.416667,4.346250,0.000000,76.008333,275.205833,279.187083,NaN
181010,84150001,2018-12-26,44.337,4.905,141.0,134.458333,1.275833,0.000000,84.525000,274.057083,276.696250,NaN
181011,84150001,2018-12-27,44.337,4.905,141.0,133.666667,0.872917,0.000833,89.170833,273.197500,275.025417,NaN
181012,84150001,2018-12-28,44.337,4.905,141.0,129.416667,3.766667,0.000833,80.737500,272.962083,276.228333,NaN
181013,84150001,2018-12-29,44.337,4.905,141.0,36.416667,7.424583,0.000000,70.754167,272.746667,277.616250,NaN
181014,84150001,2018-12-30,44.337,4.905,141.0,129.041667,8.214167,0.000000,76.116667,273.794167,277.659167,NaN
181015,84150001,2018-12-31,44.337,4.905,141.0,125.958333,7.478333,0.000000,76.900000,275.767917,279.546250,NaN


- On va ensuite vouloir associer à chaque commune la station météorologique la plus proche pour pouvoir avoir accès à la météo journalière de la commune en question. Pour cela on a besoin de calculer la distance entre la position GPS de la commune et celle de la station météo, on utilise donc cette fonction : 

In [ ]:
def distance(lat1,lon1,lat2,lon2):
    """ 
    INPUT : les coordonnées GPS de deux points
    
    OUTPUT : la distance entre ces deux points à vol d'oiseau
    """
    return 6371*np.arccos(np.sin(lat1*np.pi/180)*np.sin(lat2*np.pi/180)+np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.cos((lon2-lon1)*np.pi/180))


- La cellule suivante permet de d'obtenir pour chaque commune la station météo la plus proche en distance. Pour chaque commune on calcule la distance à chaque station et puis on séléctionne celle qui a la distance la plus faible.

In [5]:
plus_proche_station = np.zeros(len(df_communes))
for i in df_communes.index : 
    lat1 = df_communes.lat[i]
    lon1 = df_communes.lon[i]
    d_min = 100
    index_min = 0
    for j in df_stations_SE.index : 
        lat2 = df_stations_SE.lat[j]
        lon2 = df_stations_SE.lon[j]
        d = distance(lat1,lon1,lat2,lon2) 
        if d <= d_min :
            d_min = d
            index_min = j

    plus_proche_station[i] = df_stations_SE.number_sta[index_min]



- On va ensuite commencer la création du dataframe final ` df_final` en associant à chaque commune sa météo jounalière :

In [6]:
df_final = df_communes

df_final['number_sta'] = pd.DataFrame(plus_proche_station.astype(int))

df_meteo_moy = df_meteo_moy.rename(columns={'lat' : 'lat_sta', 'lon' : 'lon_sta'})

df_final = df_final.merge(df_meteo_moy, on = 'number_sta', how = 'left')

df_final['date'] = pd.to_datetime(df_final['date'])

display(df_final.head(10))


,index,code_commune_INSEE,lat,lon,region,number_sta,date,lat_sta,lon_sta,height_sta,dd,ff,precip,hu,td,t,psl
0,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-05,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,281.837879,NaN
1,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-14,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,272.222642,NaN
2,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-15,46.038,5.044,280.0,NaN,NaN,0.006667,NaN,NaN,273.077083,NaN
3,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-16,46.038,5.044,280.0,NaN,NaN,0.008333,NaN,NaN,277.585000,NaN
4,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-17,46.038,5.044,280.0,NaN,NaN,0.011667,NaN,NaN,277.890417,NaN
5,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-18,46.038,5.044,280.0,NaN,NaN,0.000833,NaN,NaN,278.067917,NaN
6,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-19,46.038,5.044,280.0,NaN,NaN,0.052500,NaN,NaN,280.643333,NaN
7,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-20,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,279.035000,NaN
8,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-21,46.038,5.044,280.0,NaN,NaN,0.025833,NaN,NaN,281.076667,NaN
9,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-22,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,283.481667,NaN


Pour finaliser notre base de donnée en vue de lui appliquer des algorithmes de régression, il nous faut maintenant ajouter une colonne "incendie" qui prend la valeur 1 s'il y a eu un incendie déclaré ce jour dans la commune et 0 sinon : 

In [ ]:
# Ajustements sur le dataframe incendie en vue de la jointure avec la base finale
df_incendies['Date de première alerte'] = pd.to_datetime(df_incendies['Date de première alerte'])
df_incendies['date'] = df_incendies['Date de première alerte'].dt.date
df_incendies = df_incendies.rename(columns={'Code INSEE' : 'code_commune_INSEE'})
df_incendies_18 = df_incendies[df_incendies['Année'] == 2018]

In [ ]:
df_final['date'] = df_final['date'].dt.date

df_final = pd.merge(df_final, df_incendies_18, how='left', on=['date', 'code_commune_INSEE'])

colonnes_a_verifier = ["Origine de l'alerte",
       'Moyens de première intervention', 'Surface parcourue (m2)',
       'Surface forêt (m2)', 'Surface maquis garrigues (m2)',
       'Autres surfaces naturelles hors forêt (m2)', 'Surfaces agricoles (m2)',
       'Autres surfaces (m2)', 'Surface autres terres boisées (m2)',
       'Surfaces non boisées naturelles (m2)',
       'Surfaces non boisées artificialisées (m2)',
       'Surfaces non boisées (m2)', 'Précision des surfaces',
       "Surface de feu à l'arrivée des secours > 0,1 ha",
       'Voie carrossable la plus proche',
       'Activité ou habitation la plus proche', 'Type de peuplement',
       'Connaissance', "Source de l'enquête", 'Nature',
       "Intervention de l'équipe RCCI", 'Décès ou bâtiments touchés',
       'Nombre de décès', 'Nombre de bâtiments totalement détruits',
       'Nombre de bâtiments partiellement détruits', 'Hygrométrie (%)',
       'Vitesse moyenne du vent (Km/h)', 'Direction du vent',
       'Température (°C)', 'Précision de la donnée',
       "Présence d'un contour valide"]  
       
# Si sur une même ligne toutes les valeurs colonnes relatives à l'incendie sont "nan" c'est qu'il n'y a eu d'incendie

df_final['incendie'] = 1-df_final[colonnes_a_verifier].isna().all(axis=1).astype(int)

display(df_final.sample(10))

-On va effectuer quelques vérifications pour voir si l'on a pas loupé des incendies par exemple :

In [ ]:
diff = list(set(df_incendies_18['code_commune_INSEE'])-set(df_final[df_final['incendie']== 1]['code_commune_INSEE']))
print("codes commune INSEE de communes ayant eu des incendies n'apparaissant pas dans la table finale :\n",diff)
print(f"Ils sont au nombre de {len(diff)}.")

Comme on peut le constater avec les deux premiers chiffres de ces codes communes INSEE qui correspondent au département, les départements où se situent ces communes  
ne sont pas des départements du Sud-Est et donc leurs feux de forêts n'entre pas dans le cadre de notre étude.  
Pour les  quelques exeptions qui restent, nous avons conclu après vérification dans la base de données de la météo qu'il s'agit en fait de ville où les données météo ne sont pas disponibles le jour de l'incendie. 